In [ ]:
import logging
#logging.basicConfig(filename='ql.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

In [ ]:
from time import sleep

In [ ]:
import datetime
import configparser

from kiteconnect import KiteTicker
from kiteconnect import KiteConnect

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os.path
from os import system

import json
import pyotp
import requests
import numpy as np
import pandas as pd
from time import sleep
import urllib.parse as urlparse

import telegram
bot = telegram.Bot('NANDJSnJNJKDKWDKFDkukf:kjskfuwfkdskufkuk')
chat_id = '@ql_march'

In [ ]:
class QLManager:
    
    global date_today
    date_today = datetime.datetime.today().date()
    
    def __init__(self,):
        self.master_contract = self.get_master_contract()
        self.banknifty_lot_size = int(self.master_contract[self.master_contract['name'] == 'BANKNIFTY'].iloc[0]['lot_size'])
        self.nifty_lot_size = int(self.master_contract[self.master_contract['name'] == 'NIFTY'].iloc[0]['lot_size'])
        self.login()
        
    def get_master_contract(self):
        date_today = datetime.datetime.today().date()
        if os.path.exists(f'master_contract_{date_today}.parquet') == True:
            master_contract = pd.read_parquet(f'master_contract_{date_today}.parquet')
            return master_contract
        else:
            try:
                while True:
                    try:
                        response = requests.get('http://api.kite.trade/instruments')
                        break
                    except:
                        print('Trying to fetch master contract again')
                        continue
                response_text = response.text
                response_text = response_text.split('\n')
                master_contract = pd.DataFrame(list(map(lambda x: x.split(','), response_text)))
                master_contract.columns = master_contract.loc[0].values
                master_contract = master_contract.iloc[1:]
                master_contract = master_contract[master_contract['tradingsymbol'].apply(lambda x: x is not None)]
                master_contract['instrument_token'] = master_contract['instrument_token'].apply(int)
                master_contract['strike'] = master_contract['strike'].apply(float)
                master_contract['name'] = master_contract['name'].apply(lambda x: str(x)[1:-1])
                master_contract['expiry'] = master_contract['expiry'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date() if x != '' else x)
                master_contract['expiry'] = np.where(master_contract['expiry'] != '',master_contract['expiry'],np.NaN)
                master_contract.sort_values('expiry', inplace=True)
                master_contract.to_parquet(f'master_contract_{date_today}.parquet')

                return master_contract

            except Exception as e:
                raise ValueError(f"Could not get master contract :: {e}")
    
    def login(self, user_id='ZTZTRPIEIE79999'):
        """
        login method can be jugaad or standard
        """
        
        config = configparser.ConfigParser()
        config.read('credentials.ini')

        api_key = config[f'kite_{user_id}']['api_key']
        api_secret = config[f'kite_{user_id}']['api_secret']
        account_username = config[f'kite_{user_id}']['account_username']
        account_password = config[f'kite_{user_id}']['account_password']
        totp = config[f'kite_{user_id}']['totp']
        auth_key = pyotp.TOTP(totp)

        kite = KiteConnect(api_key=api_key)

        if config[f'kite_{user_id}']['access_token_date'] == str(datetime.datetime.today().date()):
            access_token = config[f'kite_{user_id}']['access_token']
            print('Access Token Generated Earlier Today')

        else:
            options = Options()
            #options.add_argument('--headless')
            #options.add_argument('--disable-gpu')
            driver = webdriver.Chrome(executable_path='./chromedriver',options=options)
            driver.get(kite.login_url())
            sleep(2)
            form = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="login-form"]')))
            driver.find_element_by_xpath("//input[@type='text']").send_keys(account_username)
            driver.find_element_by_xpath("//input[@type='password']").send_keys(account_password)

            driver.find_element_by_xpath("//span[@class='su-checkbox-box']").click()
            driver.find_element_by_xpath("//button[@type='submit']").click()
            sleep(2)
            form = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="login-form"]//form')))
            driver.find_element_by_xpath("//input[@type='text']").send_keys(auth_key.now())
            driver.find_element_by_xpath("//button[@type='submit']").click()
            sleep(2)

            current_url = driver.current_url
            driver.close()

            parsed = urlparse.urlparse(current_url)
            request_token = urlparse.parse_qs(parsed.query)['request_token'][0]
            access_token = kite.generate_session(request_token=request_token,api_secret=api_secret)['access_token']

            config[f'kite_{user_id}']['access_token_date'] = str(datetime.datetime.today().date())
            config[f'kite_{user_id}']['access_token'] = access_token

            #save the config file
            with open('credentials.ini', 'w') as configfile:
                config.write(configfile)
        try:
            kite.set_access_token(access_token)
            print(f"Login Success for ID: {kite.profile()['user_id']}")
        except:
            print('Error while setting up the access_token')

        self.kite = kite
    
    def get_subscriptions(self):

        with open('subscription.json', 'r') as openfile:
            subscription = json.load(openfile)
            
        return subscription
    
    def get_ltp(self,
                instrument,
                exchange):
        '''
        returns the ltp of the instrument received
        '''
        return self.kite.ltp([f"{exchange}:{instrument}"])[f"{exchange}:{instrument}"]['last_price']


    def place_order(self,
                    tradingsymbol,
                    quantity,
                    transaction_type):#price
        '''
        place a market order for any nfo scrip
        '''

        if transaction_type == 'SELL':
            entry_id = self.kite.place_order(variety=self.kite.VARIETY_REGULAR,#variety changed to AMO
                                        tradingsymbol=tradingsymbol,
                                        exchange=self.kite.EXCHANGE_NFO,
                                        transaction_type=self.kite.TRANSACTION_TYPE_SELL,
                                        quantity=quantity,
                                        order_type=self.kite.ORDER_TYPE_MARKET,#changed
                                        product=self.kite.PRODUCT_MIS)

        elif transaction_type == 'BUY':
            entry_id = self.kite.place_order(variety=self.kite.VARIETY_REGULAR,
                                        tradingsymbol=tradingsymbol,
                                        exchange=self.kite.EXCHANGE_NFO,
                                        transaction_type=self.kite.TRANSACTION_TYPE_BUY,
                                        quantity=quantity,
                                        order_type=self.kite.ORDER_TYPE_MARKET,
                                        product=self.kite.PRODUCT_MIS)

        return entry_id
    
    def stop_loss_limit_order(self,
                              tradingsymbol,
                              buffer,
                              trigger_price,
                              quantity,
                              transaction_type):#price
        '''
        place a market order for any nfo scrip
        '''

        if transaction_type == 'SELL':
            stop_loss_order_id = self.kite.place_order(variety=self.kite.VARIETY_REGULAR,
                                                  exchange=self.kite.EXCHANGE_NFO,
                                                  tradingsymbol=tradingsymbol,
                                                  transaction_type=self.kite.TRANSACTION_TYPE_SELL,
                                                  quantity=quantity,
                                                  product=self.kite.PRODUCT_MIS,
                                                  order_type=self.kite.ORDER_TYPE_SL,
                                                  price=buffer,
                                                  validity=self.kite.VALIDITY_DAY,
                                                  trigger_price=trigger_price)

        elif transaction_type == 'BUY':
            stop_loss_order_id = self.kite.place_order(variety=self.kite.VARIETY_REGULAR,
                                                  exchange=self.kite.EXCHANGE_NFO,
                                                  tradingsymbol=tradingsymbol,
                                                  transaction_type=self.kite.TRANSACTION_TYPE_BUY,
                                                  quantity=quantity,
                                                  product=self.kite.PRODUCT_MIS,
                                                  order_type=self.kite.ORDER_TYPE_SL,
                                                  price=buffer,
                                                  validity=self.kite.VALIDITY_DAY,
                                                  trigger_price=trigger_price)



        return stop_loss_order_id
    
    def close_all_open_positions(self,):
        '''
        closes all the open positions whenever called
        '''
        positions = self.kite.positions()['net']
        for x in range(len(positions)):
            if abs(positions[x]['quantity']) > 0:
                tradingsymbol = positions[x]['tradingsymbol']
                quantity = positions[x]['quantity']
                if quantity > 0:
                    exit_id = self.kite.place_order(variety=self.kite.VARIETY_REGULAR,
                                               tradingsymbol=tradingsymbol,
                                               exchange=self.kite.EXCHANGE_NFO,
                                               transaction_type=self.kite.TRANSACTION_TYPE_SELL,
                                               quantity=quantity,
                                               order_type=self.kite.ORDER_TYPE_MARKET,
                                               product=self.kite.PRODUCT_MIS)
                elif quantity < 0:
                    exit_id = self.kite.place_order(variety=self.kite.VARIETY_REGULAR,
                                               tradingsymbol=tradingsymbol,
                                               exchange=self.kite.EXCHANGE_NFO,
                                               transaction_type=self.kite.TRANSACTION_TYPE_BUY,
                                               quantity=abs(quantity),
                                               order_type=self.kite.ORDER_TYPE_MARKET,
                                               product=self.kite.PRODUCT_MIS)
                else:
                    pass

    def cancel_all_pending_orders(self):
        '''
        cancels all the pending orders
        '''
        orders = self.kite.orders()
        for order in orders:
            if (order['status'] == 'TRIGGER PENDING') or (order['status'] == 'OPEN') | (order['status'] == 'AMO REQ RECEIVED'):
                print(order['tradingsymbol'])
                cancel_id = self.kite.cancel_order(variety=order['variety'],
                                                   order_id=order['order_id'],
                                                   parent_order_id=order['parent_order_id'])

In [ ]:
qlm = QLManager()

In [ ]:
tick = 0.05
base = 100
lot_size = 25
instrument = 'BANKNIFTY'
stoploss_pct = 40
buffer_points = 10

exchange = 'NFO'

In [ ]:
master_contract = qlm.master_contract
subscriptions = qlm.get_subscriptions()

In [ ]:
lower_bound = 100
upper_bound = 200

In [ ]:
quantity = subscriptions['ZTZTRPIEIE7365']['multiple'] * lot_size

In [ ]:
current_futures_contract = master_contract[(master_contract['instrument_type'] == 'FUT') & 
                                           (master_contract['name'] == instrument)].sort_values('expiry').iloc[0]['tradingsymbol']

In [ ]:
while datetime.datetime.now().time() < datetime.time(9,21):
    sleep(60)

In [ ]:
futures_ltp = qlm.get_ltp(current_futures_contract,exchange)

In [ ]:
atm = round(futures_ltp/base)*base

In [ ]:
while True:
    ce_strike_price = atm+1000
    # ce_strike_price = atm
    while True:
        ce_trading_symbol = master_contract[(master_contract['strike'] == ce_strike_price) & 
                                                 (master_contract['name'] == instrument) & 
                                                 (master_contract['instrument_type'] == 'CE')].iloc[0]['tradingsymbol']
        ce_ltp = qlm.get_ltp(ce_trading_symbol,'NFO')
        #print(ce_ltp)
        if (ce_ltp >=lower_bound) & (ce_ltp <=upper_bound):
            print(ce_trading_symbol,":",ce_ltp)
            break
        else:
            ce_strike_price-=100
            continue

    pe_strike_price = atm-1000
    while True:
        pe_trading_symbol = master_contract[(master_contract['strike'] == pe_strike_price) & 
                                                 (master_contract['name'] == instrument) & 
                                                 (master_contract['instrument_type'] == 'PE')].iloc[0]['tradingsymbol']
        pe_ltp = qlm.get_ltp(pe_trading_symbol,'NFO')
        #print(pe_ltp)
        if (pe_ltp >=lower_bound) & (pe_ltp <=upper_bound):
            print(pe_trading_symbol,":",pe_ltp)
            break
        else:
            pe_strike_price+=100
            continue
    break

In [ ]:
for user in subscriptions:

    if 'ql_9:20' in subscriptions[user]['subscribed_strategies']:
        
        print(f'TRADE STRATEGY FOR ACCOUNT {user}')

        ce_order_id = qlm.place_order(ce_trading_symbol,quantity,'SELL')

        pe_order_id = qlm.place_order(pe_trading_symbol,quantity,'SELL')

        for order in qlm.kite.orders():

            if order['order_id'] == ce_order_id:
                ce_traded_price = order['average_price']

            if order['order_id'] == pe_order_id:
                pe_traded_price = order['average_price']

        ce_stoploss = ce_traded_price*(1+stoploss_pct/100)
        pe_stoploss = pe_traded_price*(1+stoploss_pct/100)

        ce_stoploss = round(round(ce_stoploss/tick)*tick,2)
        pe_stoploss = round(round(pe_stoploss/tick)*tick,2)

        ce_stop_order_id = qlm.stop_loss_limit_order(ce_trading_symbol,ce_stoploss+buffer_points,ce_stoploss,quantity,'BUY')
        pe_stop_order_id = qlm.stop_loss_limit_order(pe_trading_symbol,pe_stoploss+buffer_points,pe_stoploss,quantity,'BUY')

In [ ]:
strategy = 'ql_9:20'

In [ ]:
bot.send_message(chat_id=chat_id,text=f"{strategy.upper()}\n\n{ce_trading_symbol}\n{ce_traded_price}::{ce_stoploss}\n\n{pe_trading_symbol}\n{pe_traded_price}::{pe_stoploss}")

In [ ]:
exit_time = datetime.time(14,45)

In [ ]:
while datetime.datetime.now().time() < exit_time:
    sleep(5)

In [ ]:
qlm.cancel_all_pending_orders()

In [ ]:
qlm.close_all_open_positions()

In [ ]:
mtm = 0

In [ ]:
for position in qlm.kite.positions()['net']:
    mtm = mtm+position['m2m']

In [ ]:
for user in subscriptions:
    bot.send_message(chat_id=chat_id,text=f"{user}\nSTRATS: {','.join(subscriptions[user]['subscribed_strategies'])}\n\n{mtm}")

In [ ]:
# from ql_strat import bt
# import threading

# t1 = threading.Thread(target=ql_strat)

# t1.start()

# t1.join()

In [ ]:
# logging.warning('This is a Warning')
# logging.error('NACHO')

# with open('./ql.log') as f:
#     f = f.readlines()